In [1]:
import numpy as np
from matplotlib import pyplot as plt
from IPython    import get_ipython

from GetUEFC        import UEFC
from opt_mO3        import opt_mO3
from report_opt_mO3 import report_opt_mO3

In [2]:
from time import process_time

In [3]:
def scan_ARS(tau, lam, CLdes, dbmax, n = 20):
    t1_start = process_time() 
    
    aircraft = UEFC(dbmax = dbmax, CLdes= CLdes, tau = tau, taper = lam)
    nAR = n
    nS = n
    ARarray = np.linspace(5,   15,  nAR)  # Aspect-ratio values
    Sarray  = np.linspace(0.1, 0.7, nS)   # Surface-area values (m^2)

    ARvals, Svals = np.meshgrid(ARarray, Sarray, indexing="ij")  # 2D.

    # initialize 2D output arrays
    mO3vals   = np.zeros((nAR, nS))  # Objective function (g/s^3)
    mpayvals  = np.zeros((nAR, nS))  # Payload mass (g)
    Omegavals = np.zeros((nAR, nS))  # Turn rate (rad/s)
    Rvals     = np.zeros((nAR, nS))  # Turn radius (m)
    CLvals    = np.zeros((nAR, nS))  # Lift coefficient (-)
    Tvals     = np.zeros((nAR, nS))  # Required thrust (N)
    Tmaxvals  = np.zeros((nAR, nS))  # Maximum thrust (N)
    dbvals    = np.zeros((nAR, nS))  # Wingtip deflection / wingspan
    Nvals     = np.zeros((nAR, nS))  # load factor

    for iAR,AR in enumerate(ARarray):
        for iS,S in enumerate(Sarray):

            # Determine max objective
            opt_vars, mO3, success = opt_mO3(aircraft, AR, S)

            if success:

                V = aircraft.flight_velocity(opt_vars, AR, S)

                mO3vals[iAR,iS]   = mO3
                mpayvals[iAR,iS]  = opt_vars[2]
                Omegavals[iAR,iS] = aircraft.turn_rate(opt_vars, AR, S)
                Rvals[iAR,iS]     = opt_vars[1]
                CLvals[iAR,iS]    = aircraft.lift_coefficient(opt_vars, AR, S)
                Tvals[iAR,iS]     = aircraft.required_thrust(opt_vars, AR, S)
                Tmaxvals[iAR,iS]  = aircraft.maximum_thrust(V)
                dbvals[iAR,iS]    = aircraft.wing_tip_deflection(opt_vars, AR, S)
                Nvals[iAR,iS]     = opt_vars[0]

        

        # Find and print the optimal point (where mO3 is maximized)
        mO3_opt           = np.max(mO3vals)
        (iAR_opt, iS_opt) = np.unravel_index(mO3vals.argmax(), mO3vals.shape)

        ARopt = ARvals[iAR_opt, iS_opt]
        Sopt  = Svals[iAR_opt, iS_opt]
    t1_stop = process_time()
    print("Elapsed time:", t1_stop, t1_start) 
    return mO3_opt, ARopt, Sopt, tau, lam
        
    
def scan_ARS_map(tau, lam, CLdes, dbmax, iteration, Optimal_ARS):
    iteration += 1
    """
    tau_space = np.linspace(tau[0], tau[1], num = int((tau[1]-tau[0])/.005/iteration)+1)
    lam_space = np.linspace(lam[0], lam[1], num = int((lam[1]-lam[0])/.01/iteration)+1)
    """

    tau_space = np.linspace(tau[0], tau[1], num = 5)
    lam_space = np.linspace(lam[0], lam[1], num = 5)
    
    for tau in tau_space:
        for lam in lam_space:
            mO3_opt, ARopt, Sopt, tau, lam = (scan_ARS(tau, lam, CLdes, dbmax, 20))
            if mO3_opt in Optimal_ARS:
                Optimal_ARS[mO3_opt].append([ARopt, Sopt, tau, lam])
            else:
                Optimal_ARS[mO3_opt] = [[ARopt, Sopt, tau, lam]]
    """
    
    def depth(nested_list):
        return isinstance(nested_list, list) and max(map(depth, nested_list))+1
    
    highest_scores = sorted(Optimal_ARS.keys, reverse=True)[:3]
    while tau_range < 4:
        for i in highest_scores:
            values = Optimal_ARS[i]
            depth = depth(values)
            if depth > 1:
                for i in values:
            else:
                tau_max = max(tau_max, values[0][3])
                tau_max = max(tau_max, values[0][3])
        tau_max
        tau_min
        lam_max
        lam_min
    """
    
    return Optimal_ARS



        

In [4]:
t1_start = process_time() 
Optimal_ARS = {}
Optimal_ARS_1 = scan_ARS_map((.08,.12), (.4,1), .8, .1, 0, Optimal_ARS)

t1_stop = process_time()
   
print("Elapsed time:", t1_stop, t1_start) 

C:\Users\victo\anaconda3\lib\site-packages\scipy\optimize\optimize.py:282: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "


Elapsed time: 10.609375 1.09375
Elapsed time: 21.546875 10.609375
Elapsed time: 31.328125 21.546875
Elapsed time: 42.25 31.328125
Elapsed time: 52.671875 42.25
Elapsed time: 62.53125 52.671875
Elapsed time: 72.484375 62.53125
Elapsed time: 83.390625 72.484375
Elapsed time: 94.34375 83.390625
Elapsed time: 105.4375 94.34375
Elapsed time: 114.96875 105.4375
Elapsed time: 124.953125 114.96875
Elapsed time: 134.625 124.953125
Elapsed time: 144.9375 134.625
Elapsed time: 155.484375 144.9375
Elapsed time: 164.46875 155.484375
Elapsed time: 174.28125 164.46875
Elapsed time: 185.375 174.28125
Elapsed time: 195.484375 185.375
Elapsed time: 206.546875 195.484375
Elapsed time: 214.859375 206.546875
Elapsed time: 224.03125 214.859375
Elapsed time: 235.34375 224.03125
Elapsed time: 246.03125 235.34375
Elapsed time: 256.984375 246.03125
Elapsed time: 256.984375 1.09375


In [5]:
t1_start = process_time() 
Optimal_ARS = {}
Optimal_ARS_2 = scan_ARS_map((.08,.12), (.4,1), .8, .08, 0, Optimal_ARS)

t1_stop = process_time()
   
print("Elapsed time:", t1_stop, t1_start) 

Elapsed time: 266.828125 257.015625
Elapsed time: 276.671875 266.828125
Elapsed time: 286.453125 276.671875
Elapsed time: 297.171875 286.453125
Elapsed time: 308.9375 297.171875
Elapsed time: 318.484375 308.9375
Elapsed time: 328.921875 318.484375
Elapsed time: 339.265625 328.921875
Elapsed time: 349.921875 339.265625
Elapsed time: 360.328125 349.921875
Elapsed time: 370.0625 360.328125
Elapsed time: 380.109375 370.0625
Elapsed time: 391.171875 380.109375
Elapsed time: 402.578125 391.171875
Elapsed time: 412.8125 402.578125
Elapsed time: 422.109375 412.8125
Elapsed time: 432.578125 422.109375
Elapsed time: 443.6875 432.578125
Elapsed time: 454.296875 443.6875
Elapsed time: 465.125 454.296875
Elapsed time: 473.421875 465.125
Elapsed time: 482.828125 473.421875
Elapsed time: 493.390625 482.828125
Elapsed time: 503.328125 493.390625
Elapsed time: 514.578125 503.328125
Elapsed time: 514.578125 257.0


In [6]:
t1_start = process_time() 
Optimal_ARS = {}
Optimal_ARS_2 = scan_ARS_map((.08,.12), (.4,1), .75, .1, 0, Optimal_ARS)

t1_stop = process_time()
   
print("Elapsed time:", t1_stop, t1_start) 

Elapsed time: 524.390625 514.578125
Elapsed time: 534.296875 524.390625
Elapsed time: 544.5625 534.296875
Elapsed time: 554.71875 544.5625
Elapsed time: 565.140625 554.71875
Elapsed time: 575.234375 565.140625
Elapsed time: 585.046875 575.234375
Elapsed time: 595.15625 585.046875
Elapsed time: 605.09375 595.15625
Elapsed time: 615.40625 605.09375
Elapsed time: 625.40625 615.40625
Elapsed time: 636.40625 625.40625
Elapsed time: 647.1875 636.40625
Elapsed time: 657.859375 647.1875
Elapsed time: 668.109375 657.859375
Elapsed time: 677.421875 668.109375
Elapsed time: 687.640625 677.421875
Elapsed time: 697.921875 687.640625
Elapsed time: 708.1875 697.921875
Elapsed time: 718.875 708.1875
Elapsed time: 728.25 718.875
Elapsed time: 737.609375 728.25
Elapsed time: 747.828125 737.609375
Elapsed time: 758.4375 747.828125
Elapsed time: 770.296875 758.4375
Elapsed time: 770.296875 514.578125


In [7]:
print(Optimal_ARS_1)

{93.03962897081514: [[9.210526315789473, 0.38421052631578945, 0.08, 0.4]], 78.6340729175184: [[8.157894736842106, 0.4157894736842105, 0.08, 0.55]], 63.3791065163125: [[7.105263157894736, 0.38421052631578945, 0.08, 0.7]], 46.984115275488065: [[6.052631578947368, 0.32105263157894737, 0.08, 0.85]], 33.58014475003614: [[5.526315789473684, 0.3526315789473684, 0.08, 1.0]], 85.28724825191935: [[10.263157894736842, 0.4157894736842105, 0.09, 0.4]], 73.39803487847708: [[8.68421052631579, 0.38421052631578945, 0.09, 0.55]], 58.6470480365796: [[7.631578947368421, 0.38421052631578945, 0.09, 0.7]], 45.34294744968535: [[6.578947368421053, 0.32105263157894737, 0.09, 0.85]], 33.39812239575855: [[6.052631578947368, 0.3526315789473684, 0.09, 1.0]], 76.57965795789497: [[10.789473684210526, 0.38421052631578945, 0.1, 0.4]], 66.89953093623201: [[9.210526315789473, 0.3526315789473684, 0.1, 0.55]], 55.97781299561023: [[8.157894736842106, 0.3526315789473684, 0.1, 0.7]], 42.97054207893241: [[7.105263157894736, 0.

In [10]:
highest_scores = sorted(Optimal_ARS_1.keys(), reverse=True)[:3]
print(highest_scores)

[93.03962897081514, 85.28724825191935, 78.6340729175184]


In [12]:
Optimal_ARS_1[93.03962897081514]
#ARopt, Sopt, tau, lam

[[9.210526315789473, 0.38421052631578945, 0.08, 0.4]]